<a href="https://colab.research.google.com/github/Koruvika/Koruvika.github.io/blob/master/EstimateConditionNumber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Vấn đề trong giải hệ tuyến tính

Giả sử ta cần tìm $x$ trong hệ tuyến tính: 

$Ax = b$ 

khi đã biết ma trận $A$ có kích thước mxm và vecto $b$ có độ dài m. 

Các vecto này khi lưu trong máy tính luôn tồn tại sai số trong việc làm tròn số. Vì thế hệ trong thực tế là: 
 
$\Leftrightarrow A(x + \delta x) = b + \delta b$

$\Leftrightarrow Ax + A\delta x = b + \delta b$

$\Leftrightarrow A\delta x = \delta b$

Ta xét ví dụ sau:

In [2]:
np.random.seed(42)

# tạo ra một ma trận tam giác trên 100x100
a = np.random.rand(100,100)
A = np.triu(a)

# tạo ra vecto x
x = np.random.rand(100,)

# tính b = Ax
b = A.dot(x)

# tính x_hat = A^-1 * b
x_hat = np.linalg.inv(A) * b

In [3]:
# tính infinity_norm(x_hat - x) và infinity_norm(b - A*x_hat)
norm1 = np.linalg.norm(x_hat-x, np.inf)
norm2 = np.linalg.norm(b - A.dot(x_hat), np.inf)
print(norm1, norm2)

1.44063832499248e+17 1282.3788125749993


Ở đây, ta có thể thấy rõ:

$||\hat x -x||_\infty >> ||b -A\hat x||_\infty$

Nguyên nhân gây ra vấn đề này là:
- Lỗi làm tròn số thực dấu phẩy động trong máy tính khiến $b$ trở thành $b + \delta b$.
- Trong hệ tuyến tính trên tồn tại một giá trị $\kappa(A, b, \delta b) = ||A||.||A^{-1}||$ được gọi là ***Condition Number*** của hệ tuyến tính. Và ta có quan hệ giữa sai số tương đối $\frac{\delta x}{x}$ với sai số tương đối $\frac{\delta b}{b}$ như sau:

    $\frac{\delta x}{x} \leq \kappa(A, b, \delta b) \frac{\delta b}{b}$

    (bỏ qua bước chứng minh)

    ở đây vì ta xây dựng ma trận $A$ với kích thước 100x100 theo phân phối Gaussian với $\mu = 0$ nên trên đường chéo tồn tại một giá trị xấp xỉ 0 dẫn đến $||A^{-1}||$ rất lớn, dẫn đến sai số tương đối $\frac{\delta x}{x}$ có thể rất lớn

In [4]:
normA = np.linalg.norm(A, np.inf)
normInvA = np.linalg.norm(np.linalg.inv(A), np.inf)
print("norm(A) = ", normA) 
print("normInvA(A) = ", normInvA) 
print("normA * normInvA =", normA*normInvA)

norm(A) =  51.03396146357509
normInvA(A) =  1.467655217767065e+17
normA * normInvA = 7.49002598253393e+18


# Ước tính ***Condition Number***

Không chỉ giải hệ tuyến tính, nếu có thể, ta nên cố gắng tìm Condition Number, điều này sẽ cung cấp những phản hồi hữu ích về điều kiện (độ nhạy cảm với sự thay đổi đầu vào) của vấn đề.


Dễ thấy ***Condition Number*** $\kappa(A, b, \delta b) = ||A||.||A^{-1}||$ yêu cầu tính $||A^{-1}||$ với một chi phí đắt $O(n^3)$. Điều này dẫn đến câu hỏi: "Liệu có phương pháp nào dùng để ước tính Condition Number với chi phí rẻ hơn và đạt được độ chính xác tương đối và tin cậy hay không?"

## Xây dựng giải pháp:
Đầu tiên, ta có 2 tính chất sau: 
- $||A||_\infty = max_{x \in S} ||Ax||_\infty$ với $S = \{x = \{x_0,x_1,...\} | x_i \in \{-1,1\}\}$
- $||Ax|| \leq ||A||.||x||$


$ => ||A^{-1}||_\infty = max_{x \in T} ||x||_\infty$ với $T$ là tập hợp các giá trị x thỏa mãn $Ax = b$ với $b \in S$

Ta cần xây dựng vecto $b$ sao cho $||x||_{\infty}$ là lớn nhất có thể, từ đó ta có thể xác định được $||A^{-1}||_{\infty}$

Chiến lược:
- Ta chọn $b_{m-1} = 1$. Vì ma trận A là ma trận tam giác trên, ta có thể tính được $x_{m-1}$ từ $b_{m-1}$
- Bây giờ ta có:
    
    $A_{m-2,m-2}.x_{m-2}+A_{m-2,m-1}.x_{m-1}=b_{m-2}$
    với $x_{m-1}$ đã biết và $b_{m-2}$ có thể chọn theo chiến lược, ta có thể tính được $x_{m-2}$. Chiến lược ở đây là chọn  $b_{m-2} = \{ -1, 1 \}$ sao cho $x_{m-2}$ là lớn nhất.
    
Và cứ tương tự thế... ta có thể tìm được vecto x với độ phức tạp $O(n^2)$ vậy ta tìm được $||A^{-1}||_{\infty}$ với độ phức tạp $O(n^2)$




In [5]:
print("Infinity Norm of Inverse Matrix of A: ", normInvA) 

Infinity Norm of Inverse Matrix of A:  1.467655217767065e+17


## Ví dụ minh họa

In [50]:
# tạo ma trận B có kích thước 20x20
B = np.arange(-200,200).reshape(20,20)
B = np.triu(B)
B

array([[-200, -199, -198, -197, -196, -195, -194, -193, -192, -191, -190,
        -189, -188, -187, -186, -185, -184, -183, -182, -181],
       [   0, -179, -178, -177, -176, -175, -174, -173, -172, -171, -170,
        -169, -168, -167, -166, -165, -164, -163, -162, -161],
       [   0,    0, -158, -157, -156, -155, -154, -153, -152, -151, -150,
        -149, -148, -147, -146, -145, -144, -143, -142, -141],
       [   0,    0,    0, -137, -136, -135, -134, -133, -132, -131, -130,
        -129, -128, -127, -126, -125, -124, -123, -122, -121],
       [   0,    0,    0,    0, -116, -115, -114, -113, -112, -111, -110,
        -109, -108, -107, -106, -105, -104, -103, -102, -101],
       [   0,    0,    0,    0,    0,  -95,  -94,  -93,  -92,  -91,  -90,
         -89,  -88,  -87,  -86,  -85,  -84,  -83,  -82,  -81],
       [   0,    0,    0,    0,    0,    0,  -74,  -73,  -72,  -71,  -70,
         -69,  -68,  -67,  -66,  -65,  -64,  -63,  -62,  -61],
       [   0,    0,    0,    0,    0,    

In [51]:
# thuật toán (tham lam)
# Bz = y
len = 20
z = np.zeros(len)
for i in range(len):
    row = len - 1 - i
    p = 1
    n = -1
    k = p
    for j in range(row + 1, len):
        p = p - B[row][j]*z[j]
        n = p - B[row][j]*z[j]
        if(np.abs(p) > np.abs(n)):
            k = p
        else:
            k = n
    z[row] = k/B[row][row]

In [54]:
normZ = np.linalg.norm(z, np.inf)
normInvB = np.linalg.norm(np.linalg.inv(B), np.inf)
print("Infinity Norm of Vector Z: ", normZ) 
print("Infinity Norm of Inverse Matrix of B: ", normInvB) 

Infinity Norm of Vector Z:  0.16594683059342094
Infinity Norm of Inverse Matrix of B:  0.20073653401449637
